In [1]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd, numpy as np, tensorflow as tf



In [2]:
def mean_absolute_percentage_error(y_true, y_pred):
    return tf.reduce_mean(tf.abs((y_true-y_pred) / y_true)) * 100.0

In [3]:
#df = pd.read_feather('E:/Skóli/HÍ/Vélaverkfræði Master HÍ/Lokaverkefni/Data/merged-full-25ms-24hr-28-2-24.feather')
#df = pd.read_feather('E:/Skóli/HÍ/Vélaverkfræði Master HÍ/Lokaverkefni/Data/merged-full-W-Landscape-And-Station-Elevations-25ms-24hr-11-3-24.feather')
#df = pd.read_feather('D:\Skóli\lokaverkefni_vel\data\merged-full-W-Landscape-And-Station-Elevations-25ms-24hr-11-3-24.feather')
#df = pd.read_feather('E:/Skóli/HÍ/Vélaverkfræði Master HÍ/Lokaverkefni/Data/merged-full-W-Landscape-And-Station-Elevations-25ms-24hr-13-3-24-stripped-with-klst.feather')
#df = pd.read_feather("E:/Skóli/HÍ/Vélaverkfræði Master Hí/Lokaverkefni/Data/merged-full-W-Landscape-And-Station-Elevations-Circle-larger-x2-25ms-24hr-14-3-24.feather")
#df = pd.read_feather('D:/Skóli/lokaverkefni_vel/data/merged-full-W-Landscape-And-Station-Elevations-2-sectors-25ms-24hr-18-3-24-stripped-with-klst.feather')

df = pd.read_feather("E:/Skóli/HÍ/Vélaverkfræði Master Hí/Lokaverkefni/Data/merged-full-W-Landscape-And-Station-Elevations-2-sectors-25ms-24hr-18-3-24-stripped-with-klst.feather")

df = df[df.f < df.fg]
df['gust_factor'] = df.fg / df.f

df_unfolded = df.elevations.apply(pd.Series)

df = pd.concat([df, df_unfolded], axis = 1)

df = df.dropna()
df = df.reset_index(drop = True)

In [4]:
n_elevations = df.columns[-1] + 1
df.iloc[:, -n_elevations:] = df.iloc[:, -n_elevations:].sub(df.station_elevation, axis = 0)

In [5]:
from sklearn.decomposition import PCA

df_landscape_elevation = df.iloc[:, -n_elevations:]

df_landscape_elevation = (df_landscape_elevation - df_landscape_elevation.mean()) / df_landscape_elevation.std()

n_components = 10

pca = PCA(n_components=n_components)
compressed_features = pca.fit_transform(df_landscape_elevation)

compressed_df = pd.DataFrame(data = compressed_features, columns = ['PC' + str(i) for i in range(n_components)])

df  = pd.concat([df, compressed_df], axis = 1)

In [6]:
from math import sqrt, sin, cos, acos, pi

def cornerFromCenterLand(row):
    X, Y, d = row.X, row.Y, row.d
    inlandX, inlandY = 520000, 485000

    len_v1 = sqrt((X-inlandX)**2 + (Y-inlandY)**2)

    v1 = ((X - inlandX)/len_v1, (Y - inlandY)/ len_v1)

    outX, outY = X + cos(d * pi / 180), Y + sin(d * pi / 180)

    len_v2 = sqrt(outX**2 + outY**2)

    v2 = (outX / len_v2, outY / len_v2)

    return acos(np.dot(v1, v2))
    

In [7]:
df['relativeCorner'] = df.apply(cornerFromCenterLand, axis = 1)

In [8]:
y = df.gust_factor
X = df[['Ri_01', 'Ri_12', 'Ri_02', 'N_01', 'N_12', 'N_02', 'station_elevation', 'relativeCorner'] + ['PC' + str(i) for i in range(n_components)]]

# Changing the type of X,y so as to work with Tensorflow
X, y = X.values.astype(np.float32), y.values.astype(np.float32)
scaler = StandardScaler()

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.1, random_state=42)
X_train = scaler.fit_transform(X_train)
X_val = scaler.fit_transform(X_val)
X_test = scaler.fit_transform(X_test)

In [9]:
X[0]

array([-1.1775070e+00,  2.6685779e+04,  1.8748764e+01, -8.5732063e-06,
        6.7786146e-05,  3.4937733e-05,  3.3438522e+01,  2.7327542e+00,
        2.2463312e+01,  2.7611487e+00, -8.8567799e-01, -2.7391837e+00,
        8.0440527e-01, -4.9215612e-01,  4.9296021e-01,  6.7844093e-01,
        1.8931156e-02, -7.6242559e-02], dtype=float32)

In [11]:
from tensorflow.keras.regularizers import l2

n_units = 256
n_epochs = 500
batch_size = 128
penalty = 0.1

model = tf.keras.Sequential([
    tf.keras.layers.Dense(units=n_units, activation='relu', kernel_regularizer=l2(penalty), input_shape=(X_train.shape[1],)),
    tf.keras.layers.BatchNormalization(),

    tf.keras.layers.Dense(units=n_units, activation='relu', kernel_regularizer=l2(penalty)),
    tf.keras.layers.BatchNormalization(),

    tf.keras.layers.Dense(units=n_units, activation='relu', kernel_regularizer=l2(penalty)),
    tf.keras.layers.BatchNormalization(),

    tf.keras.layers.Dense(units=n_units, activation='relu', kernel_regularizer=l2(penalty)),
    tf.keras.layers.BatchNormalization(),

    tf.keras.layers.Dense(units=n_units, activation='relu', kernel_regularizer=l2(penalty)),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dropout(0.5),

    tf.keras.layers.Dense(units=1, activation='linear')
])

In [12]:
model.compile(optimizer='adam', loss=mean_absolute_percentage_error)
# Train the model
model.fit(X_train, y_train, epochs = n_epochs, batch_size = 128, validation_data = (X_val, y_val))

Epoch 1/500
59/59 ━━━━━━━━━━━━━━━━━━━━ 5s 14ms/step - loss: 225.1927 - val_loss: 123.6386
Epoch 2/500
59/59 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - loss: 173.5785 - val_loss: 110.4870
Epoch 3/500
59/59 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 147.7630 - val_loss: 104.2302
Epoch 4/500
59/59 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 128.4312 - val_loss: 99.4821
Epoch 5/500
59/59 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 113.8919 - val_loss: 96.0515
Epoch 6/500
59/59 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - loss: 104.3592 - val_loss: 88.9564
Epoch 7/500
59/59 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 96.7661 - val_loss: 82.8659
Epoch 8/500
59/59 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 88.9521 - val_loss: 78.5784
Epoch 9/500
59/59 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - loss: 82.7677 - val_loss: 73.1489
Epoch 10/500
59/59 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - loss: 76.4178 - val_loss: 67.6529
Epoch 11/500
59/59 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 71.8177 - val_loss: 62.5679
Epoch 12/500
59/59 ━━━━━━━━━━━

In [14]:
# Evaluate the model
mape = model.evaluate(X_test, y_test)
print(f'Model evaluates to: {mape}%')

65/65 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 5.1130
Model evaluates to: 5.095616340637207%


In [ ]:
model.save(f'./saved_models/nn-{n_units}-units-huge.keras')